In [ ]:
import sys
import os
import time
import numpy as np
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import meshcat.geometry as g
import meshcat.transformations as tf
plt.style.use('seaborn-v0_8')

# Add the bsqp interface to path
sys.path.append('./build/python')

from bsqp.interface import BSQP
from force_estimator_cem import CEMForceEstimator

np.set_printoptions(linewidth=99999999)
np.random.seed(42)

urdf_path = "iiwa_description/iiwa14.urdf"
model_dir = "iiwa_description/"

class MPC_GATO:
    def __init__(self, model, N, dt, batch_size, pendulum_mass=15.0, pendulum_length=0.4, pendulum_damping=0.1):
        # Store original model for solver (without pendulum)
        self.solver_model = model
        self.solver_data = self.solver_model.createData()
        self.pendulum_mass = pendulum_mass
        self.pendulum_length = pendulum_length
        self.pendulum_damping = pendulum_damping
        
        # Create augmented model for simulation with pendulum
        self.model = self.add_pendulum_to_model(model.copy())
        self.model.gravity.linear = np.array([0, 0, -9.81])
        self.data = self.model.createData()
        
        self.solver = BSQP(model_path=urdf_path, batch_size=batch_size, N=N, dt=dt,
            max_sqp_iters=1,
            kkt_tol=0.0,
            max_pcg_iters=200,
            pcg_tol=1e-3,
            solve_ratio=1.0,
            mu=1.0,
            q_cost=5.0,
            qd_cost=2e-5,
            u_cost=2e-7,
            N_cost=50.0,
            q_lim_cost=0.0,
            vel_lim_cost=0.1,
            ctrl_lim_cost=0.0,
            rho=0.001
        )
        self.q_traj, self.q_traj_full = [], []
        
        # Dimensions - model has extra DOFs from pendulum
        self.nq_robot, self.nv_robot = self.solver_model.nq, self.solver_model.nv
        self.nq, self.nv = self.model.nq, self.model.nv  # Robot + pendulum
        self.nx = self.nq_robot + self.nv_robot  # Solver state dimension (robot only)
        self.nu = self.solver_model.nv  # Control dimension (robot only)
        self.N, self.dt, self.batch_size = N, dt, batch_size
        self.current_force_batch = None

        self.force_estimator = CEMForceEstimator(batch_size=batch_size) if batch_size > 1 else None
    
    def add_pendulum_to_model(self, model):
        """Add a 3D pendulum to the end-effector."""
        pendulum_joint_id = model.addJoint(model.njoints - 1, pin.JointModelSpherical(),pin.SE3.Identity(),"pendulum_joint") 

        # Create inertia for pendulum bob
        com = np.array([0.0, 0.0, -self.pendulum_length])  # Center of mass along -Z
        inertia_matrix = np.diag([0.001, 0.001, 0.001])  # Small inertia at COM
        pendulum_inertia = pin.Inertia(self.pendulum_mass, com, inertia_matrix)
        model.appendBodyToJoint(pendulum_joint_id, pendulum_inertia, pin.SE3.Identity())

        return model
    
    def transform_force_to_gato_frame(self, q, f_world):
        """
        Transform a force from world frame at end-effector to local frame at joint 5.
        Uses only robot joints (first nq_robot elements of q).
        """
        # Create data for solver model to do kinematics
        pin.forwardKinematics(self.solver_model, self.solver_data, q[:self.nq_robot])
        pin.updateFramePlacements(self.solver_model, self.solver_data)

        transform_world_to_ee = self.solver_data.oMi[self.solver_model.njoints - 1]
        
        force_ee_local = transform_world_to_ee.actInv(pin.Force(f_world[:3], f_world[3:]))

        result = np.zeros(6)
        result[:3] = force_ee_local.linear
        result[3:] = force_ee_local.angular

        return result
    
    def update_force_batch(self, q):
        if self.batch_size > 1:
            self.current_force_batch = self.force_estimator.generate_batch()
            self.solver.set_f_ext_B(self.current_force_batch)
    
    def evaluate_best_trajectory(self, x_last, u_last, x_curr, dt):
        """Evaluate which trajectory best matches reality"""
        if self.batch_size == 1:
            return 0
        else:
            x_next_batch = self.solver.sim_forward(x_last, u_last, dt)
            errors = np.linalg.norm(x_next_batch - x_curr[None, :], axis=1)
            best_id = np.argmin(errors)
            self.force_estimator.update(best_id, errors, batch_used=self.current_force_batch)
            return best_id
                
    def run_mpc(self, x_start, goals, sim_dt, time_per_goal):        
            
        stats = {
            'solve_times': [],
            'goal_distances': [],
            'ee_goal': [],
            'ee_actual': [],  # Actual end-effector positions
            'ee_velocity': [],  # End-effector velocities
            'controls': [],  # Control inputs (torques)
            'joint_positions': [],  # All joint positions
            'joint_velocities': [],  # All joint velocities
            'timestamps': [],  # Time stamps for each step
            'sqp_iters': [],  # SQP iterations
            'pcg_iters': [],  # PCG iterations
            'force_estimates': [],  # Force estimates (if batch)
            'force_estimates_gato': [],  # Force estimates in GATO frame
            'force_radius': [],  # Force estimator search radius
            'force_confidence': [],  # Force estimator confidence
            'best_trajectory_id': []  # Which trajectory was selected
        }
        stats['goal_outcomes_by_idx'] = ['not_reached'] * len(goals)
        
        total_sim_time, accumulated_time, steps = 0.0, 0.0, 0   
        
        x_start_aug = np.zeros(self.nq + self.nv)
        x_start_aug[:self.nx] = x_start  # Robot initial state
        x_start_aug[self.nq_robot:self.nq_robot+3] = np.array([0.3, 0.0, 0.0])  # Pendulum starts with small initial angle
        
        q, dq = x_start_aug[:self.nq], x_start_aug[self.nq:]
        
        x_curr = x_start
        x_curr_batch = np.tile(x_curr, (self.batch_size, 1))
        
        # Initialize first goal
        current_goal_idx = 0
        current_goal = goals[current_goal_idx]
        ee_g = np.tile(np.concatenate([current_goal, np.zeros(3)]), self.N)
        ee_g_batch = np.tile(ee_g, (self.batch_size, 1))
        
        XU = np.zeros(self.N*(self.nx+self.nu)-self.nu)
        XU_batch = np.tile(XU, (self.batch_size, 1))
        
        # Warm up run with initial force batch
        self.update_force_batch(q)
        solve_start = time.time()
        XU_batch, gpu_solve_time = self.solver.solve(x_curr_batch, ee_g_batch, XU_batch)
        solve_time = time.time() - solve_start
        XU_best = XU_batch[0, :]
        
        print(f"\n========== Running MPC for {time_per_goal * len(goals)} seconds with N={self.N} and batch size={self.batch_size} ==========")
        
        goal_start_time = total_sim_time
        while total_sim_time < time_per_goal * len(goals):
            steps += 1
            
            timestep = solve_time
            
            x_last = x_curr
            u_last = XU_best[self.nx:self.nx+self.nu]
            
            # ----- Step Simulation -----
            
            nsteps = int(timestep/sim_dt)
            for i in range(nsteps):
                offset = int(i/(self.dt/sim_dt)) 
                u = XU_best[self.nx+(self.nx+self.nu)*offset:(self.nx+self.nu)*(offset+1)] # get correct control input
                
                u_aug = np.zeros(self.nv)
                u_aug[:self.nu] = u
                u_aug[self.nu:] = -self.pendulum_damping * dq[self.nv_robot:] # viscous damping on pendulum
                
                q, dq = self.rk4(q, dq, u_aug, sim_dt)
                total_sim_time += sim_dt
                timestep -= sim_dt
                self.q_traj.append(q[:self.nq_robot])  # Store only robot joints
                self.q_traj_full.append(q.copy())  # Store full state
                
            accumulated_time += timestep % sim_dt 
                
            if accumulated_time - sim_dt > 0.0: # if the accumulated time is greater than sim_dt, integrate
                accumulated_time = 0.0
                
                offset = int(nsteps/(self.dt/sim_dt))
                u = XU_best[self.nx+(self.nx+self.nu)*offset:(self.nx+self.nu)*(offset+1)]
                
                u_aug = np.zeros(self.nv)
                u_aug[:self.nu] = u
                u_aug[self.nu:] = -self.pendulum_damping * dq[self.nv_robot:]
                
                q, dq = self.rk4(q, dq, u_aug, sim_dt)
                total_sim_time += sim_dt
                self.q_traj.append(q[:self.nq_robot]) 
                self.q_traj_full.append(q.copy()) 
            
            # ----- Optimize trajectory toward current goal -----
            
            current_dist, current_vel = np.linalg.norm(self.eepos(q[:self.nq_robot]) - current_goal), np.linalg.norm(dq[:self.nv_robot], ord=1)
            reached = (current_dist < 5e-2) and (current_vel < 1.0)
            timeout = (total_sim_time - goal_start_time) >= time_per_goal
            if reached or timeout:
                if reached:
                    stats['goal_outcomes_by_idx'][current_goal_idx] = 'reached'
                    print(f"\nREACHED Goal {current_goal_idx} at {total_sim_time:4.3f}s\n")
                else:
                    stats['goal_outcomes_by_idx'][current_goal_idx] = 'timeout'
                    print(f"\nTIMEOUT Goal {current_goal_idx} at {total_sim_time:4.3f}s\n")
                current_goal_idx += 1
                if current_goal_idx >= len(goals):
                    print("All goals processed")
                    break
                current_goal = goals[current_goal_idx]
                ee_g = np.tile(np.concatenate([current_goal, np.zeros(3)]), self.N)
                goal_start_time = total_sim_time
            
            x_curr = np.concatenate([q[:self.nq_robot], dq[:self.nv_robot]])
            x_curr_batch = np.tile(x_curr, (self.batch_size, 1))
            ee_g_batch[:, :] = ee_g
            XU_batch[:, :self.nx] = x_curr
            
            self.update_force_batch(q)
        #     self.solver.solver.reset_rho()
        #     self.solver.solver.reset_dual()
            solve_start = time.time()
            XU_batch_new, gpu_solve_time = self.solver.solve(x_curr_batch, ee_g_batch, XU_batch)
            solve_time = time.time() - solve_start
            best_id = self.evaluate_best_trajectory(x_last, u_last, x_curr, sim_dt)

            XU_best = XU_batch_new[best_id, :]
            XU_batch[:, :] = XU_best
            # -----
            
            ee_pos = self.eepos(q[:self.nq_robot])
            pin.forwardKinematics(self.solver_model, self.solver_model.createData(), q[:self.nq_robot], dq[:self.nv_robot])
            ee_vel = pin.getFrameVelocity(self.solver_model, self.solver_model.createData(), 6, pin.LOCAL_WORLD_ALIGNED).linear
            
            stats['timestamps'].append(total_sim_time)
            stats['solve_times'].append(float(round(gpu_solve_time/1e3, 5)))
            goaldist = np.sqrt(np.sum((ee_pos[:3] - current_goal)**2))
            stats['goal_distances'].append(float(round(goaldist, 5)))
            stats['ee_goal'].append(current_goal.copy())
            stats['ee_actual'].append(ee_pos.copy())
            stats['ee_velocity'].append(ee_vel.copy())
            stats['controls'].append(u_last.copy())
            stats['joint_positions'].append(q[:self.nq_robot].copy())
            stats['joint_velocities'].append(dq[:self.nv_robot].copy())
            stats['best_trajectory_id'].append(best_id)
            
            solver_stats = self.solver.get_stats()
            stats['sqp_iters'].append(solver_stats['sqp_iters'])
            stats['pcg_iters'].append(solver_stats['pcg_iters'][0] if len(solver_stats['pcg_iters']) > 0 else 0)

            # Calculate actual pendulum force reaction
            pendulum_com_acc = pin.getFrameClassicalAcceleration(self.model, self.data, self.model.njoints-1, pin.LOCAL_WORLD_ALIGNED).linear
            pendulum_force = -self.pendulum_mass * (pendulum_com_acc - self.model.gravity.linear)

            # Transform world-frame pendulum force at EE to the solver's GATO EE frame
            pendulum_force6_world = np.concatenate([pendulum_force, np.zeros(3)])
            pendulum_force6_gato = self.transform_force_to_gato_frame(q[:self.nq_robot], pendulum_force6_world)
        
            if steps % 512 == 0:
                if self.force_estimator:
                    est_stats = self.force_estimator.get_stats()
                    mu = est_stats['mu']
                    pf = pendulum_force6_gato
                    print(f"err=\033[91m{goaldist:4.3f}\033[0m | qd=\033[93m{current_vel:4.3f}\033[0m | t_sqp=\033[92m{gpu_solve_time/1e3:4.3f}\033[0m ms | pcg_iters={stats['pcg_iters'][-1]} | id={best_id} | f_gato=[{mu[0]:5.1f}, {mu[1]:5.1f}, {mu[2]:5.1f}] | f_p_gato=[{pf[0]:5.1f}, {pf[1]:5.1f}, {pf[2]:5.1f}] | t={total_sim_time:4.3f}s")
                else:
                    print(f"err=\033[91m{goaldist:4.3f}\033[0m | qd=\033[93m{current_vel:4.3f}\033[0m | t_sqp=\033[92m{gpu_solve_time/1e3:4.3f}\033[0m ms | pcg_iters={stats['pcg_iters'][-1]} | t={total_sim_time:4.3f}s")

        print(f"avg err: {np.mean(stats['goal_distances']):4.3f}")
        print(f"avg t_sqp: {np.mean(stats['solve_times']):4.3f}ms")
        print(f"========== MPC finished ==========")
        
        # Convert lists to numpy arrays for easier processing
        for key in stats:
            if stats[key]:
                stats[key] = np.array(stats[key])

        return self.q_traj, stats
    
    def rk4(self, q, dq, u, dt):
        k1q = dq
        k1v = pin.aba(self.model, self.data, q, dq, u)
        q2 = pin.integrate(self.model, q, k1q * dt / 2)
        k2q = dq + k1v * dt/2
        k2v = pin.aba(self.model, self.data, q2, k2q, u)
        q3 = pin.integrate(self.model, q, k2q * dt / 2)
        k3q = dq + k2v * dt/2
        k3v = pin.aba(self.model, self.data, q3, k3q, u)
        q4 = pin.integrate(self.model, q, k3q * dt)
        k4q = dq + k3v * dt
        k4v = pin.aba(self.model, self.data, q4, k4q, u)
        dq_next = dq + (dt/6) * (k1v + 2*k2v + 2*k3v + k4v)
        avg_dq = (k1q + 2*k2q + 2*k3q + k4q) / 6
        q_next = pin.integrate(self.model, q, avg_dq * dt)
        return q_next, dq_next
            
    def eepos(self, q):
        """Get end-effector position using solver model (robot only)."""
        solver_data = self.solver_model.createData()
        pin.forwardKinematics(self.solver_model, solver_data, q)
        return solver_data.oMi[self.solver_model.njoints - 1].translation

In [ ]:
goals = [
        np.array([0.6, 0.0, 0.5]),
        np.array([0.5, 0.3, 0.2]),
        np.array([0.3, 0.3, 0.8]),
        np.array([0.5, -0.5, 0.2]),
        np.array([0.0, -0.5, 0.8])
]

N = 32
dt = 0.01
sim_dt = 0.001
time_per_goal = 5.0

pendulum_mass = 15.0
pendulum_length = 0.4

x_start = np.zeros(14)

In [ ]:
model, visual_model, collision_model = pin.buildModelsFromUrdf(urdf_path, model_dir)

mpc_1 = MPC_GATO(model, N, dt, 1, pendulum_mass, pendulum_length)
q_trajectory, mpc_stats_1 = mpc_1.run_mpc(x_start, goals, sim_dt, time_per_goal)

In [ ]:
model, visual_model, collision_model = pin.buildModelsFromUrdf(urdf_path, model_dir)

mpc_32 = MPC_GATO(model, N, dt, 32, pendulum_mass, pendulum_length)
q_trajectory_32, mpc_stats_32 = mpc_32.run_mpc(x_start, goals, sim_dt, time_per_goal)

In [ ]:
# Choose which MPC instance to visualize
mpc_instance = mpc_32
goal_outcomes = list(mpc_stats_32['goal_outcomes_by_idx'])

# mpc_instance = mpc_1
# goal_outcomes = list(mpc_stats_1['goal_outcomes_by_idx'])

# ----------

model_viz, visual_model, collision_model = pin.buildModelsFromUrdf(urdf_path, model_dir)

pendulum_joint_id = model_viz.addJoint(model_viz.njoints - 1, pin.JointModelSpherical(), pin.SE3.Identity(), "pendulum_joint")
com = np.array([0.0, 0.0, -pendulum_length])
inertia_matrix = np.diag([0.001, 0.001, 0.001])
pendulum_inertia = pin.Inertia(pendulum_mass, com, inertia_matrix)
model_viz.appendBodyToJoint(pendulum_joint_id, pendulum_inertia, pin.SE3.Identity())

viz = MeshcatVisualizer(model_viz, collision_model, visual_model)
viz.initViewer(open=True)
viz.loadViewerModel(rootNodeName="robot")

viz.viewer['pendulum_rod'].set_object(g.Cylinder(height=pendulum_length, radius=0.01), g.MeshLambertMaterial(color=0x808080))
viz.viewer['pendulum_bob'].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0x0000ff))

# Default all goals to red
for i, goal in enumerate(goals):
    viz.viewer[f'goal_point_{i}'].set_object(g.Sphere(0.02), g.MeshLambertMaterial(color=0xff0000))
    T = tf.translation_matrix(np.array(goal))
    viz.viewer[f'goal_point_{i}'].set_transform(T)

# Apply colors based on outcomes: green=reached, red=timeout/not reached
for i, outcome in enumerate(goal_outcomes):
    color = 0x00ff00 if outcome == 'reached' else 0xff0000
    viz.viewer[f'goal_point_{i}'].set_object(g.Sphere(0.02), g.MeshLambertMaterial(color=color))

q_anim_full = mpc_instance.q_traj_full
data_viz = model_viz.createData()

idx = 0
while True:
    q_full = q_anim_full[idx]
    viz.display(q_full)
    
    # Update pendulum visuals
    pin.forwardKinematics(model_viz, data_viz, q_full)
    pin.updateFramePlacements(model_viz, data_viz)
    
    pendulum_frame = data_viz.oMi[pendulum_joint_id]
    rod_pos = pendulum_frame.translation + pendulum_frame.rotation @ np.array([0, 0, -pendulum_length/2])
    rot_x_90 = tf.rotation_matrix(np.pi/2, [1, 0, 0])[:3, :3]
    
    T_rod = np.eye(4)
    T_rod[:3, :3] = pendulum_frame.rotation @ rot_x_90
    T_rod[:3, 3] = rod_pos
    viz.viewer['pendulum_rod'].set_transform(T_rod)
    
    bob_pos = pendulum_frame.translation + pendulum_frame.rotation @ np.array([0, 0, -pendulum_length])
    T_bob = tf.translation_matrix(bob_pos)
    viz.viewer['pendulum_bob'].set_transform(T_bob)

    next_idx = (idx + 1) % len(q_anim_full)
    idx = next_idx
    time.sleep(0.0005)


In [ ]:
results = [{'label': "1",'stats': mpc_stats_1}, {'label': "32",'stats': mpc_stats_32}]

# Helper functions
def to_2d(array_like):
    arr = np.asarray(array_like)
    if arr.ndim == 1:
        return arr[:, None]
    return arr

def stack_rows(list_of_arrays):
    if isinstance(list_of_arrays, np.ndarray) and list_of_arrays.dtype != object:
        return list_of_arrays
    return np.vstack([np.asarray(x) for x in list_of_arrays])


In [ ]:
# 3D End-Effector Trajectory
fig = plt.figure(figsize=(7, 6))
ax = fig.add_subplot(111, projection='3d')

for res in results:
    stats = res['stats']
    ee_actual = np.asarray(stats['ee_actual'])  # (T, 3)
    ax.plot(ee_actual[:, 0], ee_actual[:, 1], ee_actual[:, 2], label=res['label'], linewidth=2)

# Plot goals
for i, goal in enumerate(goals):
    ax.scatter(goal[0], goal[1], goal[2], marker='*', s=120, c='k', label='Goal' if i == 0 else None)

ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_zlabel('Z [m]')
ax.set_title('End-Effector Trajectory (3D)')
ax.legend(loc='best')
ax.view_init(elev=25, azim=-45)
plt.tight_layout()
plt.savefig(f'ee_traj3d.png', dpi=300)
plt.show()

In [ ]:
# Tracking Error
fig, ax = plt.subplots(figsize=(8, 4))
for res in results:
    stats = res['stats']
    t = np.asarray(stats['timestamps'])
    if 'goal_distances' in stats and len(stats['goal_distances']) == len(t):
        err = np.asarray(stats['goal_distances'])
    else:
        # Fallback: compute from ee_actual and ee_goal
        ee_actual = np.asarray(stats['ee_actual'])
        ee_goal = np.asarray(stats['ee_goal'])
        err = np.linalg.norm(ee_actual[:, :3] - ee_goal[:, :3], axis=1)
    ax.plot(t, err, label=res['label'], linewidth=2)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Tracking Error [m]')
ax.set_title('End-Effector Tracking Error vs Time')
ax.grid(True, alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
plt.savefig(f'tracking_error.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
run_labels = []
reached_counts = []
timeout_counts = []
for res in results:
    stats = res['stats']
    outcomes = list(stats.get('goal_outcomes_by_idx', []))
    run_labels.append(res['label'])
    reached_counts.append(sum(1 for o in outcomes if o == 'reached'))
    timeout_counts.append(sum(1 for o in outcomes if o != 'reached'))

x = np.arange(len(run_labels))
width = 0.35
ax.bar(x - width/2, reached_counts, width, label='Reached', color='#2ca02c')
ax.bar(x + width/2, timeout_counts, width, label='Timeout/Not Reached', color='#d62728')
ax.set_xticks(x)
ax.set_xticklabels(run_labels, rotation=15)
ax.set_ylabel('Count')
ax.set_title('Goal Outcomes per Run')
ax.legend(loc='best')
plt.tight_layout()
plt.savefig(f'goal_outcomes.png', dpi=300)
plt.show()

In [ ]:
# Solve Time
fig, ax = plt.subplots(figsize=(8, 4))
for res in results:
    stats = res['stats']
    t = np.asarray(stats['timestamps'])
    solve_ms = np.asarray(stats['solve_times'])
    ax.plot(t, solve_ms, label=res['label'], linewidth=2)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Solve Time [ms]')
ax.set_title('Solve Time vs Time')
ax.grid(True, alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
plt.savefig(f'solve_time.png', dpi=300)
plt.show()

In [ ]:
# Extract time and EE positions
t1 = np.asarray(mpc_stats_1['timestamps'])
ee1 = np.asarray(mpc_stats_1['ee_actual'])  # (T1, 3)

t64 = np.asarray(mpc_stats_32['timestamps'])
ee64 = np.asarray(mpc_stats_32['ee_actual'])  # (T64, 3)

fig, axs = plt.subplots(3, 1, figsize=(9, 7), sharex=True)

axs[0].plot(t1, ee1[:, 0], label='mpc_1', linewidth=2)
axs[0].plot(t64, ee64[:, 0], label='mpc_64', linewidth=2)
axs[0].set_ylabel('X [m]')
axs[0].grid(True, alpha=0.3)
axs[0].legend(loc='best')

axs[1].plot(t1, ee1[:, 1], label='mpc_1', linewidth=2)
axs[1].plot(t64, ee64[:, 1], label='mpc_64', linewidth=2)
axs[1].set_ylabel('Y [m]')
axs[1].grid(True, alpha=0.3)
axs[1].legend(loc='best')

axs[2].plot(t1, ee1[:, 2], label='mpc_1', linewidth=2)
axs[2].plot(t64, ee64[:, 2], label='mpc_64', linewidth=2)
axs[2].set_ylabel('Z [m]')
axs[2].set_xlabel('Time [s]')
axs[2].grid(True, alpha=0.3)
axs[2].legend(loc='best')

fig.suptitle('End-Effector Trajectory Components: mpc_1 vs mpc_64', y=0.98)
plt.tight_layout()
plt.show()


In [ ]:
def segments_from_stats(stats):
    ee_goal = np.asarray(stats['ee_goal'])  # (T, 3)
    T = ee_goal.shape[0]
    if T == 0:
        return []
    change_idxs = [0]
    for i in range(1, T):
        if not np.allclose(ee_goal[i], ee_goal[i - 1], atol=1e-9):
            change_idxs.append(i)
    change_idxs.append(T)
    segments = []
    for s, e in zip(change_idxs[:-1], change_idxs[1:]):
        segments.append({'start': s, 'end': e, 'goal': ee_goal[s]})
    return segments


seg1 = segments_from_stats(mpc_stats_1)
seg32 = segments_from_stats(mpc_stats_32)
seg128 = segments_from_stats(mpc_stats_128)

num_goals = max(len(seg1), len(seg32), len(seg128))

# Prepare data arrays
EE1 = np.asarray(mpc_stats_1['ee_actual'])  # (T1, 3)
EE32 = np.asarray(mpc_stats_32['ee_actual'])  # (T32, 3)
EE128 = np.asarray(mpc_stats_128['ee_actual'])  # (T128, 3)

# Plot each goal segment individually
for i in range(num_goals):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Plot mpc_1 segment
    if i < len(seg1):
        s, e = seg1[i]['start'], seg1[i]['end']
        traj = EE1[s:e]
        ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], label='mpc_1', linewidth=2, color='red')
        goal_pt_1 = seg1[i]['goal']
    else:
        goal_pt_1 = None

    # Plot mpc_32 segment
    if i < len(seg32):
        s2, e2 = seg32[i]['start'], seg32[i]['end']
        traj2 = EE32[s2:e2]
        ax.plot(traj2[:, 0], traj2[:, 1], traj2[:, 2], label='mpc_32', linewidth=2, color='green')
        goal_pt_32 = seg32[i]['goal']
    else:
        goal_pt_32 = None

    # Plot mpc_128 segment
    if i < len(seg128):
        s3, e3 = seg128[i]['start'], seg128[i]['end']
        traj3 = EE128[s3:e3]
        ax.plot(traj3[:, 0], traj3[:, 1], traj3[:, 2], label='mpc_128', linewidth=2, color='blue')
        goal_pt_128 = seg128[i]['goal']
    else:
        goal_pt_128 = None

    goal_pt = goal_pt_1 if goal_pt_1 is not None else (goal_pt_32 if goal_pt_32 is not None else goal_pt_128)
    if goal_pt is not None:
        ax.scatter(goal_pt[0], goal_pt[1], goal_pt[2], marker='*', s=100, c='k', label='goal')
        ax.set_title(f'Goal {i}: [{goal_pt[0]:.2f}, {goal_pt[1]:.2f}, {goal_pt[2]:.2f}]')
    else:
        ax.set_title(f'Goal {i}')

    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('Z [m]')
    ax.legend(loc='best', fontsize=8)
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Plot X, Y, Z components separately for each goal segment, overlaying mpc_1, mpc_32, and mpc_128

# Use the segments_from_stats function defined in the previous cell
seg1 = segments_from_stats(mpc_stats_1)
seg32 = segments_from_stats(mpc_stats_32)
seg128 = segments_from_stats(mpc_stats_128)
num_goals = max(len(seg1), len(seg32), len(seg128))

# Full arrays
T1 = np.asarray(mpc_stats_1['timestamps'])
EE1 = np.asarray(mpc_stats_1['ee_actual'])
T32 = np.asarray(mpc_stats_32['timestamps'])
EE32 = np.asarray(mpc_stats_32['ee_actual'])
T128 = np.asarray(mpc_stats_128['timestamps'])
EE128 = np.asarray(mpc_stats_128['ee_actual'])

for gi in range(num_goals):
    fig, axs = plt.subplots(3, 1, figsize=(9, 6), sharex=False)

    goal_pt = None

    # mpc_1 segment
    if gi < len(seg1):
        s, e = seg1[gi]['start'], seg1[gi]['end']
        t_rel = T1[s:e] - T1[s]
        axs[0].plot(t_rel, EE1[s:e, 0], label='mpc_1', linewidth=1, color='red')
        axs[1].plot(t_rel, EE1[s:e, 1], label='mpc_1', linewidth=1, color='red')
        axs[2].plot(t_rel, EE1[s:e, 2], label='mpc_1', linewidth=1, color='red')
        goal_pt = seg1[gi]['goal']

    # mpc_32 segment
    if gi < len(seg32):
        s2, e2 = seg32[gi]['start'], seg32[gi]['end']
        t_rel2 = T32[s2:e2] - T32[s2]
        axs[0].plot(t_rel2, EE32[s2:e2, 0], label='mpc_32', linewidth=1, color='green')
        axs[1].plot(t_rel2, EE32[s2:e2, 1], label='mpc_32', linewidth=1, color='green')
        axs[2].plot(t_rel2, EE32[s2:e2, 2], label='mpc_32', linewidth=1, color='green')
        if goal_pt is None:
            goal_pt = seg32[gi]['goal']

    # mpc_128 segment
    if gi < len(seg128):
        s3, e3 = seg128[gi]['start'], seg128[gi]['end']
        t_rel3 = T128[s3:e3] - T128[s3]
        axs[0].plot(t_rel3, EE128[s3:e3, 0], label='mpc_128', linewidth=1, color='blue')
        axs[1].plot(t_rel3, EE128[s3:e3, 1], label='mpc_128', linewidth=1, color='blue')
        axs[2].plot(t_rel3, EE128[s3:e3, 2], label='mpc_128', linewidth=1, color='blue')
        if goal_pt is None:
            goal_pt = seg128[gi]['goal']

    if goal_pt is not None:
        axs[0].axhline(goal_pt[0], linestyle='--', color='k', alpha=0.6, label='goal')
        axs[1].axhline(goal_pt[1], linestyle='--', color='k', alpha=0.6, label='goal')
        axs[2].axhline(goal_pt[2], linestyle='--', color='k', alpha=0.6, label='goal')

    axs[0].set_ylabel('X [m]')
    axs[1].set_ylabel('Y [m]')
    axs[2].set_ylabel('Z [m]')
    axs[2].set_xlabel('Elapsed Time [s]')

    for ax in axs:
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best')

    if goal_pt is not None:
        fig.suptitle(f'Goal {gi} components: [{goal_pt[0]:.2f}, {goal_pt[1]:.2f}, {goal_pt[2]:.2f}]', y=0.98)
    else:
        fig.suptitle(f'Goal {gi} components', y=0.98)

    plt.tight_layout()
    plt.show()